In [2]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../../0_helper_functions')
import sdlnano as sdl
import calculation as calc

In [3]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2


print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


organic design space size:  10.02 Million
aqueous design space size:  1.77 Thousand
design space size:  17.74 Billion


# Check the iteration number

In [4]:
iteration = sdl.get_iteration_number()
print("This is the iteration: #", iteration)

This is the iteration: # 1


# Check the drug

In [5]:
drug = 'ACE'
print("This is the drug:", drug)

This is the drug: ACE


# Optimizer initialization (N/A)

# Generate recommendations

In [6]:
time_start = time.time()

previous_optimizer_file_name = '../iteration_' + str(iteration-1) + '/optimizer/optimizer_load_' + str(iteration-1) + '.json'
# generate recomms
ax_client_init = AxClient.load_from_json_file(previous_optimizer_file_name)
unlabeled_trial, ax_client_design = sdl.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=1)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/botorch/acquisition/multi_objective/logei.py:418: RuntimeWarning: `cache_root` is only supported for GPyTorchModels that are not MultiTask models and don't produce a TransformedPosterior. Got a model of type <class 'botorch.models.model.ModelList'>. Setting `cache_root = False`.
  NoisyExpectedHypervolumeMixin.__init__(
[INFO 07-28 15:28:10] ax.service.ax_client: Generated new trial 16 with parameters {'Drug': 67, 'SL_1': 0, 'SL_2': 0, 'SL_3': 0, 'LL_1': 0, 'LL_2': 46, 'LL_3': 71, 'P_1': 0, 'P_2': 48, 'P_3': 20, 'S_1': 80, 'S_2': 48, 'S_3': 0, 'Water': 27} using model BoTorch.
/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/ax/core/data.py:293: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude t

14 min


In [7]:
# process the trails into ratios

unlabeled_trial_processed = sdl.process_trails(unlabeled_trial)
unlabeled_trial_processed.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.0,16.000,16.000000,16.000000,16.000000,16.000000,16.0,16.000000,16.000000,15.000000,15.000000,15.000000,15.000000
mean,23.500000,0.365312,0.0,0.002,0.011125,0.016750,0.150750,0.226562,0.0,0.149687,0.077625,0.368733,0.354533,0.066667,0.210000
std,4.760952,0.174238,0.0,0.008,0.027021,0.030486,0.124506,0.187158,0.0,0.166641,0.158575,0.365099,0.345621,0.258199,0.285194
min,16.000000,0.100000,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.750000,0.265000,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,23.500000,0.353000,0.0,0.000,0.000000,0.000000,0.157000,0.257500,0.0,0.084000,0.000000,0.422000,0.310000,0.000000,0.142000
75%,27.250000,0.439000,0.0,0.000,0.000000,0.014250,0.263000,0.325500,0.0,0.304250,0.085750,0.531000,0.525000,0.000000,0.299500
max,31.000000,0.700000,0.0,0.032,0.098000,0.085000,0.324000,0.650000,0.0,0.417000,0.578000,1.000000,1.000000,1.000000,1.000000


In [8]:
unlabeled_trial
unlabeled_trial.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.0,16.0000,16.000000,16.00000,16.000000,16.000000,16.0,16.000000,16.0000,16.000000,16.000000,16.00,16.000000
mean,23.500000,71.187500,0.0,0.8125,2.562500,5.68750,34.562500,51.687500,0.0,39.062500,18.9375,42.062500,34.625000,6.25,28.250000
std,4.760952,22.109481,0.0,3.2500,5.932608,10.46084,26.941835,37.820134,0.0,43.485582,33.0746,43.040243,29.881711,25.00,35.949038
min,16.000000,28.000000,0.0,0.0000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.00,0.000000
25%,19.750000,63.750000,0.0,0.0000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.00,0.000000
50%,23.500000,71.500000,0.0,0.0000,0.000000,0.00000,46.000000,65.500000,0.0,24.000000,0.0000,37.000000,47.500000,0.00,12.500000
75%,27.250000,86.500000,0.0,0.0000,0.000000,4.25000,52.000000,81.500000,0.0,72.250000,25.7500,84.000000,55.500000,0.00,47.500000
max,31.000000,100.000000,0.0,13.0000,20.000000,30.00000,77.000000,100.000000,0.0,100.000000,100.0000,100.000000,81.000000,100.00,100.000000


In [9]:
# Convert ratios to volumes

transfer = calc.converter(unlabeled_trial_processed)
transfer.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Solvent
count,16.000000,16.000000,16.0,16.0,16.000000,16.000000,16.000000,16.000000,16.0,16.000000,16.000000,15.000000,15.000000,15.000000,15.000000,16.000000
mean,23.500000,127.859375,0.0,0.7,3.893750,5.862500,52.762500,79.296875,0.0,52.390625,27.168750,368.733333,354.533333,66.666667,210.000000,0.065625
std,4.760952,60.983449,0.0,2.8,9.457235,10.670122,43.577179,65.505436,0.0,58.324287,55.501345,365.098708,345.620532,258.198890,285.193919,0.141090
min,16.000000,35.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.750000,92.750000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,23.500000,123.550000,0.0,0.0,0.000000,0.000000,54.950000,90.125000,0.0,29.400000,0.000000,422.000000,310.000000,0.000000,142.000000,0.000000
75%,27.250000,153.650000,0.0,0.0,0.000000,4.987500,92.050000,113.925000,0.0,106.487500,30.012500,531.000000,525.000000,0.000000,299.500000,0.000000
max,31.000000,245.000000,0.0,11.2,34.300000,29.750000,113.400000,227.500000,0.0,145.950000,202.300000,1000.000000,1000.000000,1000.000000,1000.000000,0.350000


In [10]:
#dead_volume = 3.0 # mL

round(transfer[['Drug','SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000,1)

Drug       2.0
SL_1       0.0
SL_2       0.0
SL_3       0.1
LL_1       0.1
LL_2       0.8
LL_3       1.3
P_1        0.0
P_2        0.8
P_3        0.4
S_1        5.5
S_2        5.3
S_3        1.0
Water      3.2
Solvent    0.0
dtype: float64

In [11]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"

unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"

unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"

transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [12]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.to_excel(transfer_path, index = False)

# Generate new protocol

In [13]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_.py'


In [14]:
# Update the script with Excel data
updated_script = calc.update_transfer_script(template_protocol, transfer_path, output_script_path) 


In [15]:
updated_script

'protocol/iteration_1_OT_2_protocol_.py'

# Perform experiment

# Results

In [16]:
results = calc.parameters_and_complexity(iteration)
results

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Complexity,Complexity_STD
0,16,0.266,0,0.000,0.000,0.000,0.183,0.282,0,0.190,0.079,0.516,0.310,0.0,0.174,6,0
1,17,0.384,0,0.000,0.000,0.000,0.260,0.000,0,0.356,0.000,0.477,0.270,0.0,0.253,4,0
2,18,0.207,0,0.032,0.032,0.057,0.116,0.204,0,0.246,0.106,0.000,0.000,0.0,1.000,7,0
3,19,0.356,0,0.000,0.000,0.000,0.272,0.372,0,0.000,0.000,0.546,0.311,0.0,0.142,4,0
4,20,0.410,0,0.000,0.000,0.000,0.324,0.266,0,0.000,0.000,0.303,0.697,0.0,0.000,4,0
5,21,0.700,0,0.000,0.000,0.000,0.300,0.000,0,0.000,0.000,0.000,1.000,0.0,0.000,2,0
6,22,0.101,0,0.000,0.000,0.000,0.221,0.315,0,0.362,0.000,1.000,0.000,0.0,0.000,4,0
7,23,0.422,0,0.000,0.000,0.000,0.000,0.000,0,0.000,0.578,0.000,0.000,1.0,0.000,2,0
8,24,0.350,0,0.000,0.000,0.000,0.000,0.650,0,0.000,0.000,0.000,0.448,0.0,0.552,2,0
9,25,0.595,0,0.000,0.048,0.000,0.000,0.357,0,0.000,0.000,0.000,0.540,0.0,0.460,3,0


In [17]:
size_raw = calc.size_raw(iteration)
size_raw

,Data Quality,Item,Size,PD Index
0,Good,A1,231.980508,0.374270
1,Good,A2,236.372324,0.329095
2,Good,A3,245.335137,0.293648
3,Caution,A4,312.360093,0.570796
4,Good,A5,282.510319,0.461832
5,Caution,A6,295.441290,0.481025
6,Good,B1,258.033612,0.364754
7,Good,B2,243.106386,0.320841
8,Good,B3,242.000500,0.303758
9,Caution,B4,176.525646,0.309924


In [18]:
size_processed = calc.process_formulations(size_raw)
size_processed

,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,Formulation 1,237.895990,6.806446,0.332338,0.040409,1
1,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0
2,Formulation 3,247.713499,8.954568,0.329785,0.031466,1
3,Formulation 4,1000.000000,0.000000,1.000000,0.000000,0
4,Formulation 5,1000.000000,0.000000,1.000000,0.000000,0
5,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0
6,Formulation 7,1000.000000,0.000000,1.000000,0.000000,0
7,Formulation 8,285.015102,12.004342,0.315194,0.071264,1
8,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0
9,Formulation 10,1000.000000,0.000000,1.000000,0.000000,0


In [19]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,S_3,Water,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,16,0.266,0,0.000,0.000,0.000,0.183,0.282,0,0.190,...,0.0,0.174,6,0,Formulation 1,237.895990,6.806446,0.332338,0.040409,1
1,17,0.384,0,0.000,0.000,0.000,0.260,0.000,0,0.356,...,0.0,0.253,4,0,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0
2,18,0.207,0,0.032,0.032,0.057,0.116,0.204,0,0.246,...,0.0,1.000,7,0,Formulation 3,247.713499,8.954568,0.329785,0.031466,1
3,19,0.356,0,0.000,0.000,0.000,0.272,0.372,0,0.000,...,0.0,0.142,4,0,Formulation 4,1000.000000,0.000000,1.000000,0.000000,0
4,20,0.410,0,0.000,0.000,0.000,0.324,0.266,0,0.000,...,0.0,0.000,4,0,Formulation 5,1000.000000,0.000000,1.000000,0.000000,0
5,21,0.700,0,0.000,0.000,0.000,0.300,0.000,0,0.000,...,0.0,0.000,2,0,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0
6,22,0.101,0,0.000,0.000,0.000,0.221,0.315,0,0.362,...,0.0,0.000,4,0,Formulation 7,1000.000000,0.000000,1.000000,0.000000,0
7,23,0.422,0,0.000,0.000,0.000,0.000,0.000,0,0.000,...,1.0,0.000,2,0,Formulation 8,285.015102,12.004342,0.315194,0.071264,1
8,24,0.350,0,0.000,0.000,0.000,0.000,0.650,0,0.000,...,0.0,0.552,2,0,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0
9,25,0.595,0,0.000,0.048,0.000,0.000,0.357,0,0.000,...,0.0,0.460,3,0,Formulation 10,1000.000000,0.000000,1.000000,0.000000,0


In [20]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,16,0.266,0,0.000,0.000,0.000,0.183,0.282,0,0.190,...,6,0,Formulation 1,237.895990,6.806446,0.332338,0.040409,1,532.0,0
1,17,0.384,0,0.000,0.000,0.000,0.260,0.000,0,0.356,...,4,0,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
2,18,0.207,0,0.032,0.032,0.057,0.116,0.204,0,0.246,...,7,0,Formulation 3,247.713499,8.954568,0.329785,0.031466,1,414.0,0
3,19,0.356,0,0.000,0.000,0.000,0.272,0.372,0,0.000,...,4,0,Formulation 4,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
4,20,0.410,0,0.000,0.000,0.000,0.324,0.266,0,0.000,...,4,0,Formulation 5,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
5,21,0.700,0,0.000,0.000,0.000,0.300,0.000,0,0.000,...,2,0,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
6,22,0.101,0,0.000,0.000,0.000,0.221,0.315,0,0.362,...,4,0,Formulation 7,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
7,23,0.422,0,0.000,0.000,0.000,0.000,0.000,0,0.000,...,2,0,Formulation 8,285.015102,12.004342,0.315194,0.071264,1,844.0,0
8,24,0.350,0,0.000,0.000,0.000,0.000,0.650,0,0.000,...,2,0,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
9,25,0.595,0,0.000,0.048,0.000,0.000,0.357,0,0.000,...,3,0,Formulation 10,1000.000000,0.000000,1.000000,0.000000,0,0.0,0


In [21]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.237896,0.006806,0.266,0.0,0.332338,0.040409,0.500000
1,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.333333
2,1,0.247713,0.008955,0.207,0.0,0.329785,0.031466,0.583333
3,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.333333
4,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.333333
5,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.166667
6,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.333333
7,1,0.285015,0.012004,0.422,0.0,0.315194,0.071264,0.166667
8,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.166667
9,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.250000


In [22]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.237896,0.006806,0.266,0.0,0.332338,0.040409,0.500000
1,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.333333
2,1,0.247713,0.008955,0.207,0.0,0.329785,0.031466,0.583333
3,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.333333
4,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.333333
5,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.166667
6,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.333333
7,1,0.285015,0.012004,0.422,0.0,0.315194,0.071264,0.166667
8,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.166667
9,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.250000


In [23]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [24]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [25]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,16,0.266,0,0.000,0.000,0.000,0.183,0.282,0,0.190,...,0.500000,0,Formulation 1,0.237896,0.006806,0.332338,0.040409,1,0.266,0
1,17,0.384,0,0.000,0.000,0.000,0.260,0.000,0,0.356,...,0.333333,0,Formulation 2,1.000000,0.000000,1.000000,0.000000,0,0.000,0
2,18,0.207,0,0.032,0.032,0.057,0.116,0.204,0,0.246,...,0.583333,0,Formulation 3,0.247713,0.008955,0.329785,0.031466,1,0.207,0
3,19,0.356,0,0.000,0.000,0.000,0.272,0.372,0,0.000,...,0.333333,0,Formulation 4,1.000000,0.000000,1.000000,0.000000,0,0.000,0
4,20,0.410,0,0.000,0.000,0.000,0.324,0.266,0,0.000,...,0.333333,0,Formulation 5,1.000000,0.000000,1.000000,0.000000,0,0.000,0


In [26]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


In [27]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [28]:
loaded_ax_client = sdl.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)

[INFO 07-28 15:44:16] ax.service.ax_client: Completed trial 16 with data: {'Solu': (0.266, 0), 'Size': (0.237896, 0.006806), 'PDI': (0.332338, 0.040409)}.
[INFO 07-28 15:44:16] ax.service.ax_client: Completed trial 17 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 07-28 15:44:16] ax.service.ax_client: Completed trial 18 with data: {'Solu': (0.207, 0), 'Size': (0.247713, 0.008955), 'PDI': (0.329785, 0.031466)}.
[INFO 07-28 15:44:16] ax.service.ax_client: Completed trial 19 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 07-28 15:44:16] ax.service.ax_client: Completed trial 20 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 07-28 15:44:16] ax.service.ax_client: Completed trial 21 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 07-28 15:44:16] ax.service.ax_client: Completed trial 22 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 07-28 15:44:16] ax.